In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [2]:
data_path = '/kaggle/input/ham10000-preprocessed-dataset/balanced and standardized images meta for custom.csv'
image_folder = '/kaggle/input/ham10000-preprocessed-dataset/balanced_and_standardized_images/balanced_and_standardized_images'
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)

# Extract labels and encode them
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["dx"])  # Encode the 'dx' column

In [3]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.4, stratify=df['label'], random_state=42)

In [4]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 22529 validated image filenames belonging to 7 classes.
Found 5632 validated image filenames belonging to 7 classes.
Found 18774 validated image filenames belonging to 7 classes.


In [5]:
# Build DenseNet121 Model
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base model layers

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(DROPOUT_RATE)(x)
output = Dense(len(label_encoder.classes_), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss="categorical_crossentropy", metrics=["accuracy"])

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [6]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


705/705 ━━━━━━━━━━━━━━━━━━━━ 518s 711ms/step - accuracy: 0.1727 - loss: 3.5798 - val_accuracy: 0.3024 - val_loss: 1.8518
Epoch 2/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 497s 702ms/step - accuracy: 0.2459 - loss: 1.9215 - val_accuracy: 0.3331 - val_loss: 1.7942
Epoch 3/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 539s 763ms/step - accuracy: 0.2765 - loss: 1.8218 - val_accuracy: 0.3814 - val_loss: 1.7102
Epoch 4/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 515s 727ms/step - accuracy: 0.3028 - loss: 1.7651 - val_accuracy: 0.3986 - val_loss: 1.6294
Epoch 5/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 493s 696ms/step - accuracy: 0.3343 - loss: 1.6971 - val_accuracy: 0.4286 - val_loss: 1.5516
Epoch 6/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 505s 713ms/step - accuracy: 0.3473 - loss: 1.6467 - val_accuracy: 0.4636 - val_loss: 1.4965
Epoch 7/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 497s 702ms/step - accuracy: 0.3709 - loss: 1.6138 - val_accuracy: 0.4723 - val_loss: 1.4635
Epoch 8/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 507s 717ms/step - accuracy: 0.3694 - loss: 1.59

In [12]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

In [13]:
plot_model_training_curve(history)

In [14]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

587/587 ━━━━━━━━━━━━━━━━━━━━ 311s 530ms/step - accuracy: 0.5106 - loss: 1.2831

Test Accuracy: 50.82%


In [17]:
# Saving the model
model.save('/kaggle/working/dense64.keras')
model.save_weights('/kaggle/working/dense64.weights.h5')